## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import load_partition,save_partition, create_train_valid_loaders

from purrfect.training import train_model
import torch.optim as optim

from purrfect.active_learning import create_new_partition, test_model

from sklearn.model_selection import train_test_split
#from purrfect.submission import create_submission

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32
USE_AUTOCAST = True

## Definición modelo

In [4]:
class ChannelAdder(nn.Module):
    def __init__(self):
        super(ChannelAdder, self).__init__()
        # Define Sobel and Laplacian kernels as tensors
        self.sobel_x = torch.tensor([[-1., 0., 1.],
                                     [-2., 0., 2.],
                                     [-1., 0., 1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)
        
        self.sobel_y = torch.tensor([[-1., -2., -1.],
                                     [ 0.,  0.,  0.],
                                     [ 1.,  2.,  1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)

    def forward(self, x):
        epsilon = 1e-8

        # Extract the first, second, and third channels
        first_channel = x[:, 0, :, :].unsqueeze(1)  # e1 (first channel)
        second_channel = x[:, 1, :, :].unsqueeze(1)  # e2 (second channel)
        third_channel = x[:, 2, :, :].unsqueeze(1)  # error (third channel)

        # 1. Compute the first new channel: sqrt(first_channel^2 + second_channel^2)
        new_channel1 = torch.sqrt(first_channel**2 + second_channel**2)

        # 2. Compute the second new channel: 1/2 * arctan(channel2 / channel1)
        new_channel2 = 0.5 * torch.atan(second_channel / (first_channel + epsilon))

        # 3. Compute Sobel gradients and Laplacians for e1 (first_channel)
        grad_e1_x = F.conv2d(first_channel, self.sobel_x, padding=1)
        grad_e1_y = F.conv2d(first_channel, self.sobel_y, padding=1)
        grad_e1_magnitude = torch.sqrt(grad_e1_x**2 + grad_e1_y**2)

        # 4. Compute Sobel gradients and Laplacians for e2 (second_channel)
        grad_e2_x = F.conv2d(second_channel, self.sobel_x, padding=1)
        grad_e2_y = F.conv2d(second_channel, self.sobel_y, padding=1)
        grad_e2_magnitude = torch.sqrt(grad_e2_x**2 + grad_e2_y**2)

        # 5. Compute weighted ellipticity channels (e1_weighted, e2_weighted)
        #e1_weighted = first_channel / (third_channel + epsilon)
        #e2_weighted = second_channel / (third_channel + epsilon)

        # Concatenate all the channels (original and new) into the output tensor
        output = torch.cat([
            x,                 # Original 3 channels
            new_channel1,      # sqrt(channel1^2 + channel2^2)
            new_channel2,      # 1/2 * arctan(channel2 / channel1)
            grad_e1_magnitude, # Gradient magnitude of channel1
            grad_e2_magnitude, # Gradient magnitude of channel2
        ], dim=1)

        return output

In [5]:
import numpy as np
BN_MOMENTUM = 0.1
def kaiming_init(module,
                 a=0,
                 mode='fan_out',
                 nonlinearity='relu',
                 bias=0,
                 distribution='normal'):
    assert distribution in ['uniform', 'normal']
    if distribution == 'uniform':
        nn.init.kaiming_uniform_(
            module.weight, a=a, mode=mode, nonlinearity=nonlinearity)
    else:
        nn.init.kaiming_normal_(
            module.weight, a=a, mode=mode, nonlinearity=nonlinearity)
    if hasattr(module, 'bias') and module.bias is not None:
        nn.init.constant_(module.bias, bias)
class PSA_s(nn.Module):
    def __init__(self, inplanes, planes, kernel_size=1, stride=1):
        super(PSA_s, self).__init__()

        self.inplanes = inplanes
        self.inter_planes = planes // 2
        self.planes = planes
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = (kernel_size - 1) // 2
        ratio = 4

        self.conv_q_right = nn.Conv2d(self.inplanes, 1, kernel_size=1, stride=stride, padding=0, bias=False)
        self.conv_v_right = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                      bias=False)
        # self.conv_up = nn.Conv2d(self.inter_planes, self.planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.conv_up = nn.Sequential(
            nn.Conv2d(self.inter_planes, self.inter_planes // ratio, kernel_size=1),
            nn.LayerNorm([self.inter_planes // ratio, 1, 1]),
            nn.ReLU(inplace=True),
            nn.Conv2d(self.inter_planes // ratio, self.planes, kernel_size=1)
        )
        self.softmax_right = nn.Softmax(dim=2)
        self.sigmoid = nn.Sigmoid()

        self.conv_q_left = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                     bias=False)  # g
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv_v_left = nn.Conv2d(self.inplanes, self.inter_planes, kernel_size=1, stride=stride, padding=0,
                                     bias=False)  # theta
        self.softmax_left = nn.Softmax(dim=2)

        self.reset_parameters()

    def reset_parameters(self):
        kaiming_init(self.conv_q_right, mode='fan_in')
        kaiming_init(self.conv_v_right, mode='fan_in')
        kaiming_init(self.conv_q_left, mode='fan_in')
        kaiming_init(self.conv_v_left, mode='fan_in')

        self.conv_q_right.inited = True
        self.conv_v_right.inited = True
        self.conv_q_left.inited = True
        self.conv_v_left.inited = True

    def spatial_pool(self, x):
        input_x = self.conv_v_right(x)

        batch, channel, height, width = input_x.size()

        # [N, IC, H*W]
        input_x = input_x.view(batch, channel, height * width)

        # [N, 1, H, W]
        context_mask = self.conv_q_right(x)

        # [N, 1, H*W]
        context_mask = context_mask.view(batch, 1, height * width)

        # [N, 1, H*W]
        context_mask = self.softmax_right(context_mask)

        # [N, IC, 1]
        # context = torch.einsum('ndw,new->nde', input_x, context_mask)
        context = torch.matmul(input_x, context_mask.transpose(1, 2))

        # [N, IC, 1, 1]
        context = context.unsqueeze(-1)

        # [N, OC, 1, 1]
        context = self.conv_up(context)

        # [N, OC, 1, 1]
        mask_ch = self.sigmoid(context)

        out = x * mask_ch

        return out

    def channel_pool(self, x):
        # [N, IC, H, W]
        g_x = self.conv_q_left(x)

        batch, channel, height, width = g_x.size()

        # [N, IC, 1, 1]
        avg_x = self.avg_pool(g_x)

        batch, channel, avg_x_h, avg_x_w = avg_x.size()

        # [N, 1, IC]
        avg_x = avg_x.view(batch, channel, avg_x_h * avg_x_w).permute(0, 2, 1)

        # [N, IC, H*W]
        theta_x = self.conv_v_left(x).view(batch, self.inter_planes, height * width)

        # [N, IC, H*W]
        theta_x = self.softmax_left(theta_x)

        # [N, 1, H*W]
        # context = torch.einsum('nde,new->ndw', avg_x, theta_x)
        context = torch.matmul(avg_x, theta_x)

        # [N, 1, H, W]
        context = context.view(batch, 1, height, width)

        # [N, 1, H, W]
        mask_sp = self.sigmoid(context)

        out = x * mask_sp

        return out

    def forward(self, x):
        # [N, C, H, W]
        out = self.spatial_pool(x)

        # [N, C, H, W]
        out = self.channel_pool(out)

        # [N, C, H, W]
        # out = context_spatial + context_channel

        return out

class ConBnRelu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1,bias=False):
        super(ConBnRelu, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                              dilation=dilation, bias=bias)
        self.bn = nn.BatchNorm2d(out_channels, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        return out
class DoubleConv(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, use_attention=False):
        super(DoubleConv, self).__init__()
        if use_attention:
            self.seq = nn.Sequential(
                ConBnRelu(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
                PSA_s(out_channels, out_channels),
                ConBnRelu(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
            )
        else:
            self.seq = nn.Sequential(
                ConBnRelu(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
                ConBnRelu(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
            )

        self.rescale = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, x):
        return self.seq(x) + self.rescale(x)


# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        # Channel Adder
        self.channel_adder = ChannelAdder() # 3x128x128 -> 5x128x128
        # Encoder
        self.encoder1 = DoubleConv(in_channels+4, 64) # 6x128x128 -> 64x128x128
        self.encoder2 = nn.Sequential(
            nn.MaxPool2d(2), # 64x128x128 -> 64x64x64
            DoubleConv(64, 128) # 64x64x64 -> 128x64x64
        )
        
        #self.encoder3 = nn.Sequential(
        #    nn.MaxPool2d(2), # 128x64x64 -> 128x32x32
        #    DoubleConv(128, 256) # 128x32x32 -> 256x32x32
        #)
        #self.encoder4 = nn.Sequential(
        #    nn.MaxPool2d(2), # 256x32x32 -> 256x16x16
        #    DoubleConv(256, 512) # 256x16x16 -> 512x16x16
        #)

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.MaxPool2d(2), # 128x64x64 -> 128x32x32
            DoubleConv(128, 256) # 128x32x32 -> 256x32x32
        )

        # Decoder
        #self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2) # 1024x8x8 -> 512x16x16
        # concat 512x16x16 + 512x16x16 -> 1024x16x16
        #self.decoder4 = DoubleConv(1024, 512) # 1024x16x16 -> 512x16x16
        
        #self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2) # 512x16x16 -> 256x32x32
        # concat 256x32x32 + 256x32x32 -> 512x32x32
        #self.decoder3 = DoubleConv(512, 256)   # 512x32x32 -> 256x32x32
        
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2) # 256x32x32 -> 128x64x64
        # concat 128x64x64 + 128x64x64 -> 256x64x64
        self.decoder2 = DoubleConv(256, 128) # 256x64x64 -> 128x64x64
        
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2) # 128x64x64 -> 64x128x128
        # concat 64x128x128 + 64x128x128 -> 128x128x128
        self.decoder1 = nn.Sequential(
            DoubleConv(128, 64), # 128x128x128 -> 64x128x128
            nn.Conv2d(64, out_channels, kernel_size=1) # 64x128x128 -> 1x128x128
        )

    def forward(self, x):
        F.max_pool2d(x, 2)
        # Agregar features
        x = self.channel_adder(x)
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        #e3 = self.encoder3(e2)
        #e4 = self.encoder4(e3)

        # Bottleneck
        b = self.bottleneck(e2)

        # Decoder with skip connections
        #d4 = self.upconv4(b)
        #d4 = torch.cat((e4, d4), dim=1)
        #d4 = self.decoder4(d4)

        #d3 = self.upconv3(b)
        #d3 = torch.cat((e3, d3), dim=1)
        #d3 = self.decoder3(d3)

        d2 = self.upconv2(b)
        d2 = torch.cat((e2, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((e1, d1), dim=1)
        out = self.decoder1(d1)
        return out

## Creación particion inicial

In [6]:
def create_next_partitions(current_partition,k=5):#Creacion de particiones train y valid
    init_partition = []
    for i in range(k):
        init_partition += load_partition(f"partition_{k*current_partition+(i+1)}.json")
        #print(f"partition_{k*current_partition+(i+1)}.json")

    train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
    save_partition(f"partition_{current_partition+1}_train.json","partitions",train_partition)
    save_partition(f"partition_{current_partition+1}_val.json","partitions",val_partition)

In [7]:
create_next_partitions(0,1)

## Carga modelo

In [8]:
#Define model
model = UNet(3,1)
model = model.to(DEVICE)
# Define Loss
criterion = torch.nn.L1Loss()

In [9]:
sum([p.numel() for p in model.parameters() if p.requires_grad])

1948801

In [10]:
current_partition = 1
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)

In [12]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=BATCH_SIZE,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=5,
    use_autocast=USE_AUTOCAST
)

Epoch [8/50]


Validate Epoch 8: 100%|██████████| 39/39 [00:06<00:00,  6.18it/s, WMAPE=5.73, DICE=0.163, DPEAKS=115, Loss=0.0163]


Epoch [9/50]


Validate Epoch 9: 100%|██████████| 39/39 [00:06<00:00,  6.12it/s, WMAPE=2.96, DICE=0.156, DPEAKS=101, Loss=0.012]


Saving best model
Epoch [10/50]


...

Epoch [19/50]


Validate Epoch 19: 100%|██████████| 39/39 [00:06<00:00,  6.22it/s, WMAPE=6.93, DICE=0.15, DPEAKS=91.9, Loss=0.019]


Epoch [20/50]


Validate Epoch 20: 100%|██████████| 39/39 [00:06<00:00,  6.21it/s, WMAPE=5.35, DICE=0.134, DPEAKS=92.6, Loss=0.017]

early stopping: 5 epochs without improvement
Training complete.


In [13]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE*2)

Validate Epoch test: 100%|██████████| 486/486 [03:38<00:00,  2.22it/s, WMAPE=2.94, DICE=0.132, DPEAKS=80.6, Loss=0.00988]


In [11]:
create_next_partitions(current_partition)

In [12]:
current_partition = 1
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
model.load_state_dict(torch.load(best_model_path,weights_only=True))
optimizer = optim.Adam(model.parameters())


In [13]:
current_partition=2
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=BATCH_SIZE,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:13<00:00,  5.81it/s, WMAPE=4.07, DICE=0.0575, DPEAKS=39, Loss=0.0121]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:13<00:00,  5.77it/s, WMAPE=3.13, DICE=0.0565, DPEAKS=43.7, Loss=0.00952]


Saving best model
Epoch [3/50]


...

Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:14<00:00,  5.29it/s, WMAPE=2.94, DICE=0.0585, DPEAKS=45.8, Loss=0.00939]


Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:14<00:00,  5.23it/s, WMAPE=5.61, DICE=0.0714, DPEAKS=64.4, Loss=0.0174]


early stopping: 3 epochs without improvement
Training complete.


In [14]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
optimizer = optim.Adam(model.parameters())
#optimizer.load_state_dict(torch.load(last_checkpoint_path,weights_only=True)["optimizer"])
test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)
create_next_partitions(current_partition)

Validate Epoch test: 100%|██████████| 971/971 [03:04<00:00,  5.28it/s, WMAPE=1.82, DICE=0.0536, DPEAKS=39, Loss=0.00555]


In [ ]:
for current_partition in range(3,9):
    train_loader, val_loader = create_train_valid_loaders(
        f"partition_{current_partition}_train.json",
        f"partition_{current_partition}_val.json",
        "partitions",
        batch_size=BATCH_SIZE,
    )
    best_model_path = os.path.join(
        "models", f"best_model_partition_{current_partition}.pth"
    )
    last_checkpoint_path = os.path.join(
        "models", f"last_checkpoint_partition_{current_partition}.pth"
    )
    train_model(
        model,
        train_loader,
        val_loader,
        best_model_path,
        last_checkpoint_path,
        criterion,
        optimizer,
        num_epochs=50,
        device=DEVICE,
        early_stopping_patience=3,
    )
    model.load_state_dict(torch.load(best_model_path,weights_only=True))
    optimizer = optim.Adam(model.parameters())
    optimizer.load_state_dict(torch.load(last_checkpoint_path,weights_only=True)["optimizer"])
    test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)
    create_next_partitions(current_partition)

In [19]:
current_partition=3
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)

In [20]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  5.96it/s, WMAPE=2.03, DICE=0.0603, DPEAKS=61.1, Loss=0.00608]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:10<00:00,  7.13it/s, WMAPE=5.36, DICE=0.0579, DPEAKS=45, Loss=0.0157]


Epoch [3/50]


...

Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:10<00:00,  7.17it/s, WMAPE=2.04, DICE=0.0634, DPEAKS=56.6, Loss=0.00611]


Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:10<00:00,  7.17it/s, WMAPE=2.73, DICE=0.0758, DPEAKS=71.2, Loss=0.0079]


early stopping: 3 epochs without improvement
Training complete.


In [21]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: 100%|██████████| 1941/1941 [04:54<00:00,  6.59it/s, WMAPE=1.77, DICE=0.0499, DPEAKS=38.4, Loss=0.00553]


TypeError: create_next_partitions() missing 2 required positional arguments: 'model' and 'criterion'

In [23]:
create_next_partitions(current_partition)

In [24]:
current_partition=4

In [25]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  6.06it/s, WMAPE=1.51, DICE=0.0393, DPEAKS=27.1, Loss=0.00439]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.91it/s, WMAPE=1.72, DICE=0.0389, DPEAKS=28.4, Loss=0.00493]


Epoch [3/50]


...

Epoch [8/50]


Validate Epoch 8: 100%|██████████| 77/77 [00:10<00:00,  7.07it/s, WMAPE=1.45, DICE=0.0379, DPEAKS=27.2, Loss=0.00424]


Epoch [9/50]


Validate Epoch 9: 100%|██████████| 77/77 [00:11<00:00,  6.50it/s, WMAPE=1.41, DICE=0.038, DPEAKS=26.9, Loss=0.00424]


early stopping: 3 epochs without improvement
Training complete.


In [26]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition)

Validate Epoch test: 100%|██████████| 1941/1941 [04:50<00:00,  6.68it/s, WMAPE=1.41, DICE=0.0383, DPEAKS=28.1, Loss=0.00421]


In [27]:
current_partition=5

In [10]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:09<00:00,  7.86it/s, WMAPE=1.87, DICE=0.0453, DPEAKS=46.4, Loss=0.00513]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:10<00:00,  7.59it/s, WMAPE=1.97, DICE=0.0453, DPEAKS=44, Loss=0.00521]


Epoch [3/50]


Validate Epoch 3: 100%|██████████| 77/77 [00:09<00:00,  8.01it/s, WMAPE=2.45, DICE=0.0461, DPEAKS=44, Loss=0.00671]


Epoch [4/50]


Validate Epoch 4: 100%|██████████| 77/77 [00:09<00:00,  7.87it/s, WMAPE=1.92, DICE=0.0455, DPEAKS=45.8, Loss=0.00514]


early stopping: 3 epochs without improvement
Training complete.


In [11]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_6_val: 100%|██████████| 77/77 [00:10<00:00,  7.21it/s, WMAPE=1.65, DICE=0.0436, DPEAKS=36.1, Loss=0.00477]


In [12]:
current_partition=6

In [13]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:09<00:00,  8.14it/s, WMAPE=2.05, DICE=0.046, DPEAKS=52.4, Loss=0.00538]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:09<00:00,  8.04it/s, WMAPE=2.06, DICE=0.0462, DPEAKS=52.8, Loss=0.00534]


Saving best model
Epoch [3/50]


...

Epoch [14/50]


Validate Epoch 14: 100%|██████████| 77/77 [00:09<00:00,  8.05it/s, WMAPE=2.14, DICE=0.0452, DPEAKS=49.8, Loss=0.00559]


Epoch [15/50]


Validate Epoch 15: 100%|██████████| 77/77 [00:09<00:00,  8.12it/s, WMAPE=2.04, DICE=0.046, DPEAKS=50.1, Loss=0.00524]


early stopping: 3 epochs without improvement
Training complete.


In [14]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_7_val: 100%|██████████| 77/77 [00:10<00:00,  7.23it/s, WMAPE=1.54, DICE=0.043, DPEAKS=33.5, Loss=0.00457]
